<a href="https://colab.research.google.com/github/ryosuke1224-haas/RNN/blob/main/Recurrent_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Language Model

In [1]:
!pip install tensorflow
!pip install keras
!pip install h5py

In [2]:
s='Sing a song of sixpence,\
A pocket full of rye.\
Four and twenty blackbirds,\
Baked in a pie.\
When the pie was opened\
The birds began to sing;\
Wasn’t that a dainty dish,\
To set before the king.\
The king was in his counting house,\
Counting out his money;\
The queen was in the parlour,\
Eating bread and honey.\
The maid was in the garden,\
Hanging out the clothes,\
When down came a blackbird\
And pecked off her nose.'

with open('rhymes.txt','w') as f:
  f.write(s)

    Sing a song of sixpence,
    A pocket full of rye.
    Four and twenty blackbirds,
    Baked in a pie.

    When the pie was opened
    The birds began to sing;
    Wasn’t that a dainty dish,
    To set before the king.

    The king was in his counting house,
    Counting out his money;
    The queen was in the parlour,
    Eating bread and honey.

    The maid was in the garden,
    Hanging out the clothes,
    When down came a blackbird
    And pecked off her nose.

In [3]:
#load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [4]:
#load text
raw_text = load_doc('rhymes.txt')
print(raw_text)

# clean
tokens = raw_text.split()
raw_text = ' '.join(tokens)

# organize into sequences of characters
length = 10 #Tried 5,10,15
sequences = list()
for i in range(length, len(raw_text)):
    # select sequence of tokens
    seq = raw_text[i-length:i+1]
    # store
    sequences.append(seq)
print('Total Sequences: %d' % len(sequences))

Sing a song of sixpence,A pocket full of rye.Four and twenty blackbirds,Baked in a pie.When the pie was openedThe birds began to sing;Wasn’t that a dainty dish,To set before the king.The king was in his counting house,Counting out his money;The queen was in the parlour,Eating bread and honey.The maid was in the garden,Hanging out the clothes,When down came a blackbirdAnd pecked off her nose.
Total Sequences: 384


In [5]:
# save sequences to file
out_filename = 'char_sequences.txt'
save_doc(sequences, out_filename)

# Train a Model

In [6]:
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [7]:
# load

in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')

In [8]:
# integer encode sequences of characters
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))
sequences = list()
for line in lines:
    encoded_seq = [mapping[char] for char in line]
    sequences.append(encoded_seq)

vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)

Vocabulary Size: 38


In [9]:
# Define model
from keras.layers import Dropout

model = Sequential()
model.add(LSTM(100, input_shape=(X.shape[1], X.shape[2]))) 
model.add(Dropout(0.5)) 
model.add(Dense(vocab_size, activation='softmax')) 
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
history=model.fit(X, y, epochs=150)  

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               55600     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 38)                3838      
Total params: 59,438
Trainable params: 59,438
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/150
12/12 [==============================] - 2s 9ms/step - loss: 3.6353 - accuracy: 0.0239
Epoch 2/150
12/12 [==============================] - 0s 8ms/step - loss: 3.5648 - accuracy: 0.0894
Epoch 3/150
12/12 [==============================] - 0s 8ms/step - loss: 3.3728 - accuracy: 0.1416
Epoch 4/150
12/12 [==============================] - 0s 8ms/step - loss: 3.3059 

In [10]:
model.save('model.h5')
dump(mapping, open('mapping.pkl', 'wb'))

# Generating Text

In [11]:
from pickle import load
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    for _ in range(n_chars):
        encoded = [mapping[char] for char in in_text]
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        encoded = to_categorical(encoded, num_classes=len(mapping))
        yhat = model.predict_classes(encoded, verbose=0)
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        in_text += char
    return in_text

model = load_model('model.h5')
mapping = load(open('mapping.pkl', 'rb'))

In [12]:
print(generate_seq(model, mapping, 10, 'Sing a son', 20))
print(generate_seq(model, mapping, 10, 'king was i', 20))
print(generate_seq(model, mapping, 10, 'hello worl', 20))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Sing a song of sixpence,A pock
king was in his counting house
hello worl,,Fsrdd rrnwdttetat 


In [13]:
history=model.fit(X, y, epochs=150,validation_split=0.3,verbose=0) 
model.save('model.h5')
dump(mapping, open('mapping.pkl', 'wb'))

def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    for _ in range(n_chars):
        encoded = [mapping[char] for char in in_text]
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        encoded = to_categorical(encoded, num_classes=len(mapping))
        yhat = model.predict_classes(encoded, verbose=0)
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        in_text += char
    return in_text

model = load_model('model.h5')
mapping = load(open('mapping.pkl', 'rb'))
print(generate_seq(model, mapping, 10, 'Sing a son', 20))
print(generate_seq(model, mapping, 10, 'king was i', 20))
print(generate_seq(model, mapping, 10, 'hello worl', 20))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Sing a song of sixpence,A pock
king was in his counting house
hello worls,snnddnnoo teeheoos


# Train a word-level model

In [14]:
s="Venus and Adonis [But, lo! from forth a copse]\
William Shakespeare - 1564-1616\
But, lo! from forth a copse that neighbours by,\
A breeding jennet, lusty, young, and proud,\
Adonis’ trampling courser doth espy,\
And forth she rushes, snorts and neighs aloud;\
The strong-neck’d steed, being tied unto a tree,\
Breaketh his rein, and to her straight goes he.\
Imperiously he leaps, he neighs, he bounds,\
And now his woven girths he breaks asunder;\
The bearing earth with his hard hoof he wounds,\
Whose hollow womb resounds like heaven’s thunder;\
The iron bit he crushes ’tween his teeth\
Controlling what he was controlled with.\
His ears up-prick’d; his braided hanging mane\
Upon his compass'd crest now stand on end;\
His nostrils drink the air, and forth again,\
As from a furnace, vapours doth he send:\
His eye, which scornfully glisters like fire,\
Shows his hot courage and his high desire.\
Sometime her trots, as if he told the steps,\
With gentle majesty and modest pride;\
Anon he rears upright, curvets and leaps,\
As who should say, ’Lo! thus my strength is tried;\
And this I do to captivate the eye\
Of the fair breeder that is standing by.’\
What recketh he his rider’s angry stir,\
His flattering ’Holla,’ or his ’Stand, I say?’\
What cares he now for curb of pricking spur?\
For rich caparisons or trapping gay?\
He sees his love, and nothing else he sees,\
Nor nothing else with his proud sight agrees.\
Look, when a painter would surpass the life, \
In limning out a well-proportion’d steed,\
His art with nature’s workmanship at strife,\
As if the dead the living should exceed;\
So did this horse excel a common one,\
In shape, in courage, colour, pace and bone\
Round-hoof’d, short-jointed, fetlocks shag and long,\
Broad breast, full eye, small head, and nostril wide,\
High crest, short ears, straight legs and passing strong,\
Thin mane, thick tail, broad buttock, tender hide:\
Look, what a horse should have he did not lack,\
Save a proud rider on so proud a back.\
Sometimes he scuds far off, and there he stares;\
Anon he starts at stirring of a feather;\
To bid the wind a race he now prepares,\
And whe’r he run or fly they know not whether;\
For through his mane and tail the high wind sings,\
Fanning the hairs, who wave like feather’d wings.\
He looks upon his love, and neighs unto her;\
She answers him as if she knew his mind;\
Being proud, as females are, to see him woo her,\
She puts on outward strangeness, seems unkind,\
Spurns at his love and scorns the heat he feels,\
Beating his kind embracements with her heels.\
Then, like a melancholy malcontent,\
He vails his tail that, like a falling plume\
Cool shadow to his melting buttock lent:\
He stamps, and bites the poor flies in his fume.\
His love, perceiving how he is enrag’d,\
Grew kinder, and his fury was assuag’d.\
His testy master goeth about to take him;\
When lo! the unback’d breeder, full of fear,\
Jealous of catching, swiftly doth forsake him,\
With her the horse, and left Adonis there.\
As they were mad, unto the wood they hie them,\
Out-stripping crows that strive to over-fly them.\
I prophesy they death, my living sorrow,\
If thou encounter with the boar to-morrow.\
But if thou needs wilt hunt, be rul’d by me;\
Uncouple at the timorous flying hare,\
Or at the fox which lives by subtlety,\
Or at the roe which no encounter dare:\
Pursue these fearful creatures o’er the downs,\
And on they well-breath’d horse keep with they hounds.\
And when thou hast on food the purblind hare,\
Mark the poor wretch, to overshoot his troubles\
How he outruns with winds, and with what care\
He cranks and crosses with a thousand doubles:\
The many musits through the which he goes\
Are like a labyrinth to amaze his foes.\
Sometime he runs among a flock of sheep,\
To make the cunning hounds mistake their smell,\
And sometime where earth-delving conies keep,\
To stop the loud pursuers in their yell,\
And sometime sorteth with a herd of deer;\
Danger deviseth shifts; wit waits on fear:\
For there his smell with other being mingled,\
The hot scent-snuffing hounds are driven to doubt,\
Ceasing their clamorous cry till they have singled \
With much ado the cold fault cleanly out;\
Then do they spend their mouths: Echo replies,\
As if another chase were in the skies.\
By this, poor Wat, far off upon a hill,\
Stands on his hinder legs with listening ear,\
To hearken if his foes pursue him still:\
Anon their loud alarums he doth hear;\
And now his grief may be compared well\
To one sore sick that hears the passing-bell.\
Then shalt thou see the dew-bedabbled wretch\
Turn, and return, indenting with the way;\
Each envious briar his weary legs doth scratch,\
Each shadow makes him stop, each murmur stay:\
For misery is trodden on by many,\
And being low never reliev’d by any.\
Lie quietly, and hear a little more;\
Nay, do not struggle, for thou shalt not rise:\
To make thee hate the hunting of the boar,\
Unlike myself thou hearst’ me moralize,\
Applying this to that, and so to so;\
For love can comment upon every woe."

with open('shakespear_venues_and_Adonis.txt','w') as f:
  f.write(s)

In [15]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np
tokenizer = Tokenizer()

data = open('shakespear_venues_and_Adonis.txt').read()
texts = data.lower().split("\n")
tokenizer.fit_on_texts(texts)
total_words = len(tokenizer.word_index) + 1

sequences = []
for line in texts:
 token_list = tokenizer.texts_to_sequences([line])[0]
 for i in range(1, len(token_list)):
  n_gram_sequence = token_list[:i+1]
  sequences.append(n_gram_sequence)

max_length = max([len(x) for x in sequences])
sequences = np.array(pad_sequences(sequences, maxlen=max_length, padding='pre'))
sequences

array([[  0,   0,   0, ...,   0, 118,   2],
       [  0,   0,   0, ..., 118,   2,  58],
       [  0,   0,   0, ...,   2,  58,  59],
       ...,
       [  0,   0, 118, ..., 464, 465,  53],
       [  0, 118,   2, ..., 465,  53, 466],
       [118,   2,  58, ...,  53, 466, 467]], dtype=int32)

In [16]:
def generate_padded_sequences(sequences):
    max_length = max([len(x) for x in sequences])
    sequences = np.array(pad_sequences(sequences, maxlen=max_length, padding='pre'))
    
    predictors, label = sequences[:,:-1],sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_length

predictors, label, max_length = generate_padded_sequences(sequences)

In [17]:
def create_model(max_length, total_words):
    input_len = max_length - 1
    model = Sequential()
    
    model.add(Embedding(total_words, 50, input_length=input_len))
    model.add(LSTM(200))
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')   
    return model

model = create_model(max_length, total_words)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 885, 50)           23400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               200800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 468)               94068     
Total params: 318,268
Trainable params: 318,268
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
28/28 [==============================] - 53s 2s/step - loss: 6.1100
Epoch 2/100
28/28 [==============================] - 51s 2s/step - loss: 5.7553
Epoch 3/100
28/28 [==============================] - 51s 2s/step - loss: 5.6351
Epoch 4/100
28/28 [==============================] - 51s 2s/step - loss: 5.6256
Epoch 5/100
28/28 [==============================] - 50s 2s/step - loss: 5.6508
Epoch 6/100
28/28 [==============================] - 51s 2s/step - loss: 5.5653
Epoch 7/100
28/28 [==============================] - 57s 2s/step - loss: 5.5540
Epoch 8/100
28/28 [==============================] - 51s 2s/step - loss: 5.4555
Epoch 9/100
28/28 [==============================] - 51s 2s/step - loss: 5.3791
Epoch 10/100
28/28 [==============================] - 51s 2s/step - loss: 5.2241
Epoch 11/100
28/28 [==============================] - 50s 2s/step - loss: 5.1456
Epoch 12/100
28/28 [==============================] - 50s 2s/step - loss: 4.8628
Epoch 13/100
28/28 [=================

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [ ]:
#Original sentence
print(generate_text('Then, like a melancholy malcontent',10,model,10)) 
print('Oritinal text is "Then, like a melancholy malcontent,He vails his tail that, like a falling plume"')
print('===============================')
print(generate_text('For misery is trodden on by many,',10,model,10))
print('Oritinal text is "For misery is trodden on by many,And being low never reliev’d by any"')
print('===============================')

#sentence from other Shakespeare's work. One line from Macbeth
print(generate_text('This is the sergeant,',10,model,10))